In [ ]:
import os
print("HÜCRE 1: 64x64 Projesi Hazırlanıyor")

# 1. Drive Bağlantısı
from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. Gerekli Kütüphaneler
print("\nKütüphaneler kuruluyor...")
!pip install click scipy requests imageio pillow psutil tqdm ninja torch torchvision opensimplex

# 3. Kod Deposu
if not os.path.exists("/content/stylegan2-ada-pytorch"):
    print("\nKod deposu klonlanıyor...")
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch.git

# Çalışma dizinini ayarla
os.chdir("/content/stylegan2-ada-pytorch")
print(f"\nBaşarıyla şu klasöre geçildi: {os.getcwd()}")

HÜCRE 1: 64x64 Projesi Hazırlanıyor
Mounted at /content/drive

Kütüphaneler kuruluyor...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.0/268.0 kB 13.9 MB/s eta 0:00:00

Kod deposu klonlanıyor...
Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 577, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 577 (delta 21), reused 17 (delta 17), pack-reused 546 (from 2)
Receiving objects: 100% (577/577), 8.43 MiB | 16.22 MiB/s, done.
Resolving deltas: 100% (329/329), done.

Başarıyla şu klasöre geçildi: /content/stylegan2-ada-pytorch


In [ ]:
import os
import zipfile
import io
from PIL import Image
from tqdm import tqdm

print("HÜCRE 2: Veri Seti 64x64'e Dönüştürülüyor")

# Yollar
kaynak_zip = "/content/drive/MyDrive/TUBITAK_Projesi/dataset_temiz_v1.zip"
hedef_zip = "/content/empresyonizm_64.zip"
TARGET_SIZE = (64, 64) # Hedef boyut

# Veri Dönüştürme
if not os.path.exists(kaynak_zip):
    print("HATA: Kaynak veri seti bulunamadı! Drive yollarını kontrol edin.")
else:
    print(f"Veri seti işleniyor: {TARGET_SIZE} piksel...")
    try:
        with zipfile.ZipFile(kaynak_zip, 'r') as zf_in:
            with zipfile.ZipFile(hedef_zip, 'w', compression=zipfile.ZIP_STORED) as zf_out:
                file_list = [f for f in zf_in.namelist() if not f.startswith('__') and f.lower().endswith(('.png', '.jpg', '.jpeg'))]

                for filename in tqdm(file_list):
                    try:
                        with zf_in.open(filename) as f:
                            img = Image.open(f).convert('RGB')
                            # 64x64'e küçült
                            img_resized = img.resize(TARGET_SIZE, Image.LANCZOS)

                            img_byte_arr = io.BytesIO()
                            img_resized.save(img_byte_arr, format='PNG')
                            zf_out.writestr(os.path.basename(filename).split('.')[0] + ".png", img_byte_arr.getvalue())
                    except:
                        pass
        print(f"\nVeri seti hazır: {hedef_zip}")
    except Exception as e:
        print(f"Hata: {e}")

HÜCRE 2: Veri Seti 64x64'e Dönüştürülüyor
Veri seti işleniyor: (64, 64) piksel...


 70%|███████   | 481/683 [00:29<00:08, 25.08it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (171668516 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|██████████| 683/683 [00:45<00:00, 15.13it/s]


Veri seti hazır: /content/empresyonizm_64.zip


In [ ]:
import os

print("HÜCRE 3: 64x64 Eğitim SIFIRDAN Başlatılıyor")

# Ayarlar
OUTDIR = "/content/drive/MyDrive/TUBITAK_Projesi/results_64_FINAL"
DATA = "/content/empresyonizm_64.zip"

#Sıfırdan başlıyoruz

GPUS = 1
KIMG = 2500      # Bu hedefe çok çabuk ulaşacaksınız
GAMMA = 1        # Düşük çözünürlük için
AUGPIPE = "bgc"
METRICS = "none"

# --- 64px İÇİN HIZ AYARLARI ---
BATCH_SIZE = 64  # 64px için 64 batch size (Çok hızlı)
WORKERS = 2
SNAP = 25        # Her 25 kimg'de bir kayıt al (Gelişimi anlık gör)
# ------------------------------

print(f"Veri: {DATA}")
print(f"Çıktı: {OUTDIR}")
print(f"Hız: Batch Size {BATCH_SIZE}")

!python train.py --outdir={OUTDIR} \
                 --data={DATA} \
                 --gpus={GPUS} \
                 --kimg={KIMG} \
                 --gamma={GAMMA} \
                 --batch={BATCH_SIZE} \
                 --augpipe={AUGPIPE} \
                 --metrics={METRICS} \
                 --workers={WORKERS} \
                 --snap={SNAP}

Görüntülenen çıkış son 5000 satıra kısaltıldı.
ModuleNotFoundError: No module named 'upfirdn2d_plugin'

  warnings.warn('Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:\n\n' + traceback.format_exc())
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
/content/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/content/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/stylegan2-ada-pytorch/torch_utils/custom_ops.py", line 111, in get_plugin
    module = importlib.import_module(module_name)
             ^^^^^^^^^^^^^